In [ ]:
import numpy as np
import pandas as pd
import requests
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Javascript, display
from ipywidgets import widgets 
from ipywidgets import interact
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from collections import defaultdict
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib.pyplot import figure
#from IPython.display import Image
from datetime import datetime
import ipywidgets as widgets # widget install
from IPython.display import display #widget install
from IPython.display import clear_output
#from IPython.utils import utils
#output = widgets.Output()

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
from IPython.display import Javascript, display
from ipywidgets import widgets 
from ipywidgets import interact

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Restart everything")
button.on_click(run_all)
display(button)

In [ ]:
df = pd.read_csv('MP.csv', skiprows=1)

df["Union"]=df["Union"].astype('str')
df.Orders_Trade = df.Orders_Trade.astype('str')
df.ClassificationRequested = df.ClassificationRequested.astype('str')
df["RequiredDate"] = pd.to_datetime(df["RequiredDate"])
df["A&_Test_Booked"] = pd.to_datetime(df["A&_Test_Booked"])
df["Fall Arrest Date"] = pd.to_datetime(df["Fall Arrest Date"])
df["Orientation Date"] = pd.to_datetime(df["Orientation Date"])
df["LOTTO Date"] = pd.to_datetime(df["LOTTO Date"])
df["Confined Space Date"] = pd.to_datetime(df["Confined Space Date"])
df["Aerial Work Date"] = pd.to_datetime(df["Aerial Work Date"])

#A&D test booked
df['Requried to A&T booked'] = df['A&_Test_Booked'] - df['RequiredDate'] 
df['Requried to A&T booked'] = df['Requried to A&T booked']/np.timedelta64(1,'D')

#Fall Arrest Date
df['Requried to Fall'] = df['Fall Arrest Date'] - df['RequiredDate']
df['Requried to Fall'] = df['Requried to Fall']/np.timedelta64(1,'D')

#Aerial Work Date
df['Requried to Aerial'] = df['Aerial Work Date'] - df['RequiredDate']
df['Requried to Aerial'] = df['Requried to Aerial']/np.timedelta64(1,'D')

#Lotto Date
df['Requried to Lotto'] = df['LOTTO Date'] - df['RequiredDate'] 
df['Requried to Lotto'] = df['Requried to Lotto']/np.timedelta64(1,'D')

#Confined Space 
df['Requried to Conf Space'] = df['Confined Space Date'] - df['RequiredDate']
df['Requried to Conf Space'] = df['Requried to Conf Space']/np.timedelta64(1,'D')

#Orientation Date
df['Requried to Orient Date'] = df['Orientation Date'] - df['RequiredDate'] 
df['Requried to Orient Date'] = df['Requried to Orient Date']/np.timedelta64(1,'D')

df=df.fillna(0)


In [ ]:
output = widgets.Output()

ALL = 'ALL'

def unique_sorted_values_plus_ALL(array): 
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

common_filter = pd.DataFrame()

dropdown_trade  = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.Orders_Trade),description="Trade")
dropdown_union = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.Union),description="Union")
dropdown_skill   = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.ClassificationRequested),description="Skill")

def common_filtering (trade, union, skill):
    global common_filter
    output.clear_output()
    
    if (trade == ALL) & (union == ALL) & (skill == ALL):
        common_filter = df
    
    elif (trade == ALL) & (union == ALL):
        common_filter = df[df.ClassificationRequested == skill]

    elif (trade == ALL) & (skill==ALL):
        common_filter= df[df.Union == union]
        
    elif (skill == ALL) & (union == ALL):
        common_filter= df[df.Orders_Trade == trade]
    
    elif (union == ALL)& (trade != ALL) & (skill != ALL):
        common_filter = df[(df.Orders_Trade == trade) & (df.ClassificationRequested == skill)]   
    elif (skill == ALL) & (trade != ALL) & (union != ALL):
        common_filter = df[(df.Orders_Trade == trade) & (df.Union == union)]
    
    elif (trade == ALL) & (skill != ALL) & (union !=ALL) :
        common_filter = df[(df.ClassificationRequested == skill) & df.Union == union]
    
    else:
        common_filter = df[(df.Orders_Trade == trade) & (df.Union == union) & (df.ClassificationRequested == skill)]
        
    with output:
        display(common_filter)
        

if len(common_filter) == 0:
    common_filter = df

else:
    common_filter = common_filter
    

def dropdown_trade_eventhandler(change):
    common_filtering(change.new, dropdown_union.value, dropdown_skill.value)

def dropdown_union_eventhandler(change):
    common_filtering(dropdown_trade.value, change.new, dropdown_skill.value)

def dropdown_skill_eventhandler(change):
    common_filtering(dropdown_trade.value, dropdown_union.value, change.new)


dropdown_trade.observe(dropdown_trade_eventhandler, names='value')
dropdown_union.observe(dropdown_union_eventhandler, names='value')
dropdown_skill.observe(dropdown_skill_eventhandler, names='value')

#display(dropdown_trade)
#display(dropdown_union)
#display(dropdown_skill)

print("Please select at least one dropdown and press the Refresh button below")

In [ ]:
widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.Orders_Trade),description="Trade")

In [ ]:
widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.Union),description="Union")

In [ ]:
widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.ClassificationRequested),description="Skill")

In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Refresh")
button.on_click(run_all)
display(button)

In [ ]:
df2=common_filter

dflook=df2[['Requried to A&T booked',
       'Requried to Fall', 'Requried to Aerial', 'Requried to Lotto',
       'Requried to Conf Space', 'Requried to Orient Date']]
dflook = dflook.fillna(0)
dflook = dflook.dropna(thresh=1)
dflook.astype('timedelta64[D]')
print()

# Using Standard Scaler to Scale Data
scaler = StandardScaler()
#X=scaler.fit_transform(df) 
X=scaler.fit_transform(dflook) 
dflook.shape

# Loop through kmeans using 2-12 clusters, default estimators set to 100
index_dict = defaultdict(list)
index_fi = defaultdict(list)
for index in range(2,12,2):
    Kmean = KMeans(n_clusters=index)
    Kmean.fit(X)
    target_vals = pd.Series(Kmean.labels_)
    y = target_vals
    
    clf=ExtraTreesClassifier(n_estimators=100,
                              random_state=0)
    clf = clf.fit(X,y)
    importances = clf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    index_fi[index] = np.sort(importances)[::-1]
    index_dict[index]=indices

# Create a dict with the values to plot.
new_dict = defaultdict(list)
for k in range(0,5):
    for val in list(index_dict.keys()):
        pos = np.where(index_dict[val]==k) # Index position for value equal to k
        new_dict[k].append(12-pos[0][0])
        
#Plotting theimportance of features based on 2 clusters. 
column_names = dflook.columns
plt.figure(figsize=(20,15))
plt.bar(column_names[index_dict[2]],  index_fi[2])
plt.xlabel('Time Between Events', fontsize=18)
plt.ylabel('Feature Importance Ranking (Percentage of importance)',fontsize=18)
plt.title('Importance Score of Features using 2 clusters', fontsize=18)
plt.xticks(rotation=90, fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
cols_order = column_names[index_dict[2]]


# correlation matrix for time
Correlation=dflook.corr()
plt.figure(figsize=(15, 15))
sns.heatmap(Correlation,annot=True)

# correlation matrix for all
Correlation=df.corr()
plt.figure(figsize=(15, 15))
sns.heatmap(Correlation,annot=True)